# Lille Metropole Dataset - Qualité de l'air

https://fr.wikipedia.org/wiki/Indice_de_qualit%C3%A9_de_l%27air

Le nouvel indice de qualité de l’air ATMO est un indicateur journalier gradué de 1 (bon) à 6 (extrêmement mauvais) qui permet de caractériser de manière simple et globale la qualité de l’air d’une agglomération urbaine.

Il se compose de 5 sous-indices, chacun étant représentatif d'un polluant de l'air :
- dioxyde d'azote (NO2 ou code_no2)
- dioxyde de soufre (SO2 ou code_so2)
- ozone (O3 ou code_o3)
- particules fines de moins de 10mm (PM10 ou code_pm10)
- particules fines de moins de 2.5mm (PM2.5 ou code_pm25)

Le sous-indice le plus élevé détermine l'indice du jour.

Il est calculé à partir des mesures des stations représentatives de la pollution de fond. Il ne prend pas en compte les phénomènes de proximité (automobile ou industriel).

| O3         | SO2       | NO2       | PM10      | PM2.5   | Niveau              |
|------------|-----------|-----------|-----------|---------|---------------------|
| 0 à 50     | 0 à 100   | 0 à 40    | 0 à 20    | 0 à 10  | Bon                 |
| 50 à 100   | 100 à 200 | 40 à 90   | 20 à 40   | 10 à 20 | Moyen               |
| 100 à 130  | 200 à 350 | 90 à 120  | 40 à 50   | 20 à 25 | Dégradé             |
| 130 à 240  | 350 à 500 | 120 à 230 | 50 à 100  | 25 à 50 | Mauvais             |
| 240 à 380  | 500 à 750 | 230 à 340 | 100 à 150 | 50 à 75 | Très mauvais        |
| > 380      | > 750     | > 340     | > 150     | > 75    | Extrêmement mauvais |	 	 	 	 	

## 1. Setting-Up

#### 1.1 Importing librairies & modules

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import time # provides various time-related functions
from datetime import datetime
import geopandas as gpd
# from arcgis.features import FeatureLayer

#### 1.2. Creating the relevant folders and paths

In [126]:
# Defining current folder as our main directory
dirname = os.getcwd()

#Getting current date
datestr = time.strftime("%Y-%m-%d")

# location folders variables
data_in = dirname + "\\da_data_raw\\" + datestr + "\\"
data_out = dirname + "\\da_data_workfiles\\" + datestr + "\\"

  
# checking if data_in path is not present, then create it.
if not os.path.exists(data_in):
    os.makedirs(data_in)
    print(f"Directory Created:{data_in}\n")
    
# checking if data_out path is not present then create it.
if not os.path.exists(data_out):
    os.makedirs(data_out)
    print(f"Directory Created:{data_out}\n")
    

print(f"Main directory: {dirname}")
print(f"Raw data folder: {data_in}")
print(f"Analysis and workfile folder: {data_out}")

Main directory: c:\Users\33671\Documents\Python\MEL_air-quality
Raw data folder: c:\Users\33671\Documents\Python\MEL_air-quality\da_data_raw\2022-10-07\
Analysis and workfile folder: c:\Users\33671\Documents\Python\MEL_air-quality\da_data_workfiles\2022-10-07\


#### 1.3 Verifying the api response

In [127]:
#alternative source: https://data-atmo-hdf.opendata.arcgis.com/search?collection=Dataset
#alternative source: https://services8.arcgis.com/rxZzohbySMKHTNcy/ArcGIS/rest/services/ind_hdf_2021/FeatureServer/0
dataset_id = "indice-qualite-de-lair"
format = "json"
limit = "10"
r = requests.get(f"https://opendata.lillemetropole.fr/api/v2/catalog/datasets/{dataset_id}/exports/{format}?limit={limit}", 
                 timeout=2)

print(f"URL: {r.url}")
print(f"HTTP Response Status Code: {r.status_code}") 
print(f"HTTP Error: {r.raise_for_status()}")
print(f"Encoding: {r.encoding}")
print(f"Header content type: {r.headers.get('content-type')}")
print(f"Cookies: {r.cookies}")

r.close()


URL: https://opendata.lillemetropole.fr/api/v2/catalog/datasets/indice-qualite-de-lair/exports/json?limit=10
HTTP Response Status Code: 200
HTTP Error: None
Encoding: utf-8
Header content type: application/json; charset=utf-8
Cookies: <RequestsCookieJar[]>


#### 1.4 Data Collection

In [128]:
dataset_id = "indice-qualite-de-lair"
format = "json"
limit = "-1" # argument to pass to get the full dataset 
df_raw = pd.read_json(f"https://opendata.lillemetropole.fr/api/v2/catalog/datasets/{dataset_id}/exports/{format}?limit={limit}")

#### 1.5 Getting a first feel of the data

In [130]:
df_raw.head()

,date_ech,code_qual,lib_qual,coul_qual,date_dif,source,type_zone,code_zone,lib_zone,code_no2,...,code_pm25,x_wgs84,y_wgs84,x_reg,y_reg,epsg_reg,objectid,geo_shape,geo_point_2d,code_posta
0,2022-02-02T01:00:00+00:00,2,Moyen,#50CCAA,2022-02-03T14:10:07+00:00,Atmo HDF,commune,59386,MARQUETTE LEZ LILLE,1,...,1,3.05917,50.67413,704190,7064143,2154,118631,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 3.059169999999999, 'lat': 50.67413}",59520
1,2022-02-02T01:00:00+00:00,2,Moyen,#50CCAA,2022-02-03T14:10:07+00:00,Atmo HDF,commune,59410,MONS EN BAROEUL,1,...,1,3.10817,50.64327,707664,7060707,2154,118654,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 3.10817, 'lat': 50.64327}",59370
2,2022-02-02T01:00:00+00:00,2,Moyen,#50CCAA,2022-02-03T14:10:07+00:00,Atmo HDF,commune,59421,MOUVAUX,1,...,1,3.13669,50.70408,709673,7067487,2154,118664,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 3.13669, 'lat': 50.70408}",59420
3,2022-02-02T01:00:00+00:00,2,Moyen,#50CCAA,2022-02-03T14:10:07+00:00,Atmo HDF,commune,59458,PERONNE EN MELANTOIS,1,...,1,3.17061,50.56429,712107,7051915,2154,118701,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 3.17061, 'lat': 50.56429}",59273
4,2022-02-02T01:00:00+00:00,2,Moyen,#50CCAA,2022-02-03T14:10:07+00:00,Atmo HDF,commune,59470,PREMESQUES,1,...,1,2.94864,50.66056,696361,7062630,2154,118712,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.94864, 'lat': 50.66056}",59840


In [131]:
df_raw.tail()

,date_ech,code_qual,lib_qual,coul_qual,date_dif,source,type_zone,code_zone,lib_zone,code_no2,...,code_pm25,x_wgs84,y_wgs84,x_reg,y_reg,epsg_reg,objectid,geo_shape,geo_point_2d,code_posta
17495,2022-07-07T02:00:00+00:00,2,Moyen,#50CCAA,2022-07-08T14:10:07+00:00,Atmo HDF,commune,59566,SEQUEDIN,1,...,1,2.98184,50.62491,698713,7058656,2154,694733,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.98184, 'lat': 50.62490999999999}",59320
17496,2022-07-07T02:00:00+00:00,2,Moyen,#50CCAA,2022-07-08T14:10:07+00:00,Atmo HDF,commune,59611,VERLINGHEM,1,...,1,3.00037,50.68240,700025,7065063,2154,694778,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 3.00037, 'lat': 50.68240000000001}",59237
17497,2022-07-07T02:00:00+00:00,2,Moyen,#50CCAA,2022-07-08T14:10:07+00:00,Atmo HDF,commune,59650,WATTRELOS,1,...,1,3.21667,50.70555,715333,7067664,2154,694815,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 3.21667, 'lat': 50.70555}",59150
17498,2022-07-08T02:00:00+00:00,3,Dégradé,#F0E641,2022-07-09T14:10:07+00:00,Atmo HDF,commune,59051,BASSEE,1,...,1,2.80943,50.54002,686469,7049214,2154,698024,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.80943, 'lat': 50.54001999999999}",59480
17499,2022-07-08T02:00:00+00:00,3,Dégradé,#F0E641,2022-07-09T14:10:07+00:00,Atmo HDF,commune,59052,BAUVIN,1,...,1,2.89324,50.51585,692416,7046510,2154,698025,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.89324, 'lat': 50.51584999999999}",59221


In [132]:
df_raw.sample(5)

,date_ech,code_qual,lib_qual,coul_qual,date_dif,source,type_zone,code_zone,lib_zone,code_no2,...,code_pm25,x_wgs84,y_wgs84,x_reg,y_reg,epsg_reg,objectid,geo_shape,geo_point_2d,code_posta
11981,2022-01-11T01:00:00+00:00,2,Moyen,#50CCAA,2022-01-12T14:10:07+00:00,Atmo HDF,commune,59025,AUBERS,2,...,2,2.81985,50.59596,687223,7055445,2154,38714,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.81985, 'lat': 50.595960000000005}",59249
17320,2022-06-28T02:00:00+00:00,2,Moyen,#50CCAA,2022-06-29T14:10:07+00:00,Atmo HDF,commune,59658,WICRES,1,...,1,2.86827,50.56911,690652,7052447,2154,660722,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.86827, 'lat': 50.56911}",59134
10049,2022-06-10T02:00:00+00:00,2,Moyen,#50CCAA,2022-06-11T14:10:07+00:00,Atmo HDF,commune,59252,FRELINGHIEN,1,...,1,2.95157,50.70296,696572,7067356,2154,595917,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.9515700000000002, 'lat': 50.70296}",59236
7079,2022-02-13T01:00:00+00:00,2,Moyen,#50CCAA,2022-02-14T14:10:08+00:00,Atmo HDF,commune,59196,ENNETIERES EN WEPPES,1,...,2,2.93254,50.64039,695219,7060384,2154,160129,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.93254, 'lat': 50.64039}",59320
6376,2022-02-14T01:00:00+00:00,2,Moyen,#50CCAA,2022-02-15T14:10:10+00:00,Atmo HDF,commune,59636,WAMBRECHIES,1,...,1,3.04705,50.69595,703329,7066574,2154,164342,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 3.047049999999999, 'lat': 50.695949999...",59118


#### 1.6 Exporting raw data to CSV

In [181]:
# generating a raw csv file with timestamp included in the name
timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
file_name = "air-quality-index_" + timestr + ".csv"
print(f"file name: {file_name}")

df_raw.to_csv(data_in + file_name, index = False)

file name: air-quality-index_2022-10-07_16-45-48.csv


## 2. Preparing Data for Analysis

#### 2.1 Basic exploration

In [134]:
# data.info has the merits of combining many functions together
# - see if there's null values replacing `df.isnull().sum()`
# - see the dtype of each colymns replacing `df.dtypes`
# - see the shape of the dataframe replacing `df.shape`
# - estimating the memory usage replacing `df.memory_usage`
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17500 entries, 0 to 17499
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date_ech      17500 non-null  object 
 1   code_qual     17500 non-null  int64  
 2   lib_qual      17500 non-null  object 
 3   coul_qual     17500 non-null  object 
 4   date_dif      17500 non-null  object 
 5   source        17500 non-null  object 
 6   type_zone     17500 non-null  object 
 7   code_zone     17500 non-null  int64  
 8   lib_zone      17500 non-null  object 
 9   code_no2      17500 non-null  int64  
 10  code_so2      17500 non-null  int64  
 11  code_o3       17500 non-null  int64  
 12  code_pm10     17500 non-null  int64  
 13  code_pm25     17500 non-null  int64  
 14  x_wgs84       17500 non-null  float64
 15  y_wgs84       17500 non-null  float64
 16  x_reg         17500 non-null  int64  
 17  y_reg         17500 non-null  int64  
 18  epsg_reg      17500 non-nu

In [135]:
# quick overview of date_ech to see the formatting used
df_raw["date_ech"].value_counts(ascending=True)

2022-07-08T02:00:00+00:00    20
2022-02-02T01:00:00+00:00    95
2022-07-01T02:00:00+00:00    95
2022-07-02T02:00:00+00:00    95
2022-05-06T02:00:00+00:00    95
                             ..
2022-03-16T01:00:00+00:00    95
2022-03-17T01:00:00+00:00    95
2022-03-18T01:00:00+00:00    95
2022-03-05T01:00:00+00:00    95
2022-05-25T02:00:00+00:00    95
Name: date_ech, Length: 185, dtype: int64

In [136]:
# quick overview of date_dif to see the formatting used
df_raw["date_dif"].value_counts(ascending=True)

2022-07-09T14:10:07+00:00    20
2022-02-03T14:10:07+00:00    95
2022-07-02T14:10:07+00:00    95
2022-07-03T14:10:07+00:00    95
2022-05-07T14:10:06+00:00    95
                             ..
2022-03-17T14:10:07+00:00    95
2022-03-18T14:10:06+00:00    95
2022-03-19T14:10:06+00:00    95
2022-03-06T14:10:08+00:00    95
2022-05-26T14:10:07+00:00    95
Name: date_dif, Length: 185, dtype: int64

In [137]:
# checking how many unique "lib_zone" (i.e. cities) we observe each day in this dataframe
df_raw["lib_zone"].nunique()

95

In [138]:
df_raw["lib_zone"].value_counts(ascending=True)

MARQUETTE LEZ LILLE    184
HEM                    184
PERENCHIES             184
LINSELLES              184
HALLUIN                184
                      ... 
COMINES                185
CAPINGHEM              185
BOUSBECQUE             185
BOIS GRENIER           185
CARNIN                 185
Name: lib_zone, Length: 95, dtype: int64

In [139]:
# creating a copy of the dataframe
df = df_raw.copy()

#### 2.2 Dealing with duplicates

When I started this project, I was getting duplicates (days with 190 occurences of the same date instead of 95). However, it's not the case anymore. I'm still keeping this **2.2 Duplicates** section in case it happens again when we query the API in the future. 

Performing a `df.duplicated().sum()` gives me a *TypeError: unhashable type: 'dict'* error. This is because columns **geo_shape** and **geo_point_2d** are dictionaries. I'll exclude them. 

In [140]:
# check duplication of df excluding geo_shape and geo_point_2d
df.duplicated(df.columns.difference(['geo_shape', 'geo_point_2d'])).sum()

0

I found this **0** result to be strange given my initial observations with dates **value_counts** and decided to proceed further by including columns one by one to see if duplicates emerge

In [141]:
#I used to get 817 duplicates but not anymore. Keeping the code just in case it happens again. 
df.duplicated(['date_ech', 'date_dif', 'code_zone', 'lib_zone', 'code_qual', 'lib_qual', 'coul_qual', 
               'source', 'type_zone', 'code_no2', 'code_so2', 'code_o3', 'code_pm10', 'code_pm25', 
               'x_wgs84', 'y_wgs84', 'x_reg', 'y_reg', 'epsg_reg']).sum()

0

In [142]:
df.duplicated(['date_ech', 'date_dif', 'code_zone', 'lib_zone', 'code_qual', 'lib_qual', 'coul_qual', 
               'source', 'type_zone', 'code_no2', 'code_so2', 'code_o3', 'code_pm10', 'code_pm25', 'x_wgs84', 
               'y_wgs84', 'x_reg', 'y_reg', 'epsg_reg', 'objectid']).sum()

0

There are (were) numerous duplicates if we ignore **objectid**. Since all other variables are similar and we expect one observation per day per city, I will remove **objectid**, then remove duplicates. 

In [143]:
# removeing objectid column
df.drop(columns=['objectid'], inplace=True)

# converting **geo_shape** and **geo_point_2d** to string
df['geo_shape'] = df.geo_shape.astype(str)
df['geo_point_2d'] = df.geo_point_2d.astype(str)

In [144]:
df.duplicated().sum()

0

In [145]:
df.drop_duplicates(inplace=True)

#### 2.3 Removing and rearranging columns

In [146]:
# Removing
df.drop(columns=['coul_qual', 'date_dif', 'source', 'type_zone', 'code_zone', 'x_wgs84', 'y_wgs84', 'x_reg', 'y_reg', 'epsg_reg', 'geo_shape', 'geo_point_2d'], 
        errors='ignore', 
        inplace=True)

df.head()

,date_ech,code_qual,lib_qual,lib_zone,code_no2,code_so2,code_o3,code_pm10,code_pm25,code_posta
0,2022-02-02T01:00:00+00:00,2,Moyen,MARQUETTE LEZ LILLE,1,1,2,1,1,59520
1,2022-02-02T01:00:00+00:00,2,Moyen,MONS EN BAROEUL,1,1,2,1,1,59370
2,2022-02-02T01:00:00+00:00,2,Moyen,MOUVAUX,1,1,2,1,1,59420
3,2022-02-02T01:00:00+00:00,2,Moyen,PERONNE EN MELANTOIS,1,1,2,1,1,59273
4,2022-02-02T01:00:00+00:00,2,Moyen,PREMESQUES,1,1,2,1,1,59840


In [147]:
# Renaming columns
cols = ['date', 'quality_code', 'quality_label', 'city', 'no2', 'so2', 'o3', 'pm10', 'pm2-5', 'zip_code']
df.columns = cols

df.head()

,date,quality_code,quality_label,city,no2,so2,o3,pm10,pm2-5,zip_code
0,2022-02-02T01:00:00+00:00,2,Moyen,MARQUETTE LEZ LILLE,1,1,2,1,1,59520
1,2022-02-02T01:00:00+00:00,2,Moyen,MONS EN BAROEUL,1,1,2,1,1,59370
2,2022-02-02T01:00:00+00:00,2,Moyen,MOUVAUX,1,1,2,1,1,59420
3,2022-02-02T01:00:00+00:00,2,Moyen,PERONNE EN MELANTOIS,1,1,2,1,1,59273
4,2022-02-02T01:00:00+00:00,2,Moyen,PREMESQUES,1,1,2,1,1,59840


In [148]:
# Moving columns
city = df.pop('city')
df.insert(1, 'city', city)
del city

zip_code = df.pop('zip_code')
df.insert(2, 'zip_code', zip_code)
del zip_code

quality_label = df.pop('quality_label')
df.insert(3, 'quality_label', quality_label)
del quality_label

df.head()

,date,city,zip_code,quality_label,quality_code,no2,so2,o3,pm10,pm2-5
0,2022-02-02T01:00:00+00:00,MARQUETTE LEZ LILLE,59520,Moyen,2,1,1,2,1,1
1,2022-02-02T01:00:00+00:00,MONS EN BAROEUL,59370,Moyen,2,1,1,2,1,1
2,2022-02-02T01:00:00+00:00,MOUVAUX,59420,Moyen,2,1,1,2,1,1
3,2022-02-02T01:00:00+00:00,PERONNE EN MELANTOIS,59273,Moyen,2,1,1,2,1,1
4,2022-02-02T01:00:00+00:00,PREMESQUES,59840,Moyen,2,1,1,2,1,1


#### 2.4 Convertions

In [149]:
# lower cases
df['quality_label'] = df['quality_label'].str.lower()
df['city'] = df['city'].str.lower()

In [150]:
df.dtypes

date             object
city             object
zip_code          int64
quality_label    object
quality_code      int64
no2               int64
so2               int64
o3                int64
pm10              int64
pm2-5             int64
dtype: object

In [151]:
#convert date 
df['date'] = pd.to_datetime(df['date']).dt.date
df["date"] = pd.to_datetime(df["date"]).dt.normalize() 

df.head()

,date,city,zip_code,quality_label,quality_code,no2,so2,o3,pm10,pm2-5
0,2022-02-02,marquette lez lille,59520,moyen,2,1,1,2,1,1
1,2022-02-02,mons en baroeul,59370,moyen,2,1,1,2,1,1
2,2022-02-02,mouvaux,59420,moyen,2,1,1,2,1,1
3,2022-02-02,peronne en melantois,59273,moyen,2,1,1,2,1,1
4,2022-02-02,premesques,59840,moyen,2,1,1,2,1,1


In [152]:
df.dtypes

date             datetime64[ns]
city                     object
zip_code                  int64
quality_label            object
quality_code              int64
no2                       int64
so2                       int64
o3                        int64
pm10                      int64
pm2-5                     int64
dtype: object

#### 2.5 Enriching data

To facilitate data manipulation later on, let's add some usefull date related columns

In [171]:
#using index since date column is my index
df['year'] = pd.DatetimeIndex(df.date).year
df['month'] = pd.DatetimeIndex(df.date).month
df['day'] = pd.DatetimeIndex(df.date).day
df['weekday'] = pd.DatetimeIndex(df.date).weekday #The day of the week with Monday=0, Sunday=6.
df['week_of_year'] = df.date.dt.isocalendar().week

#sorting  by date then city
df.sort_values(by=['date', 'city'], inplace=True)

df.head()

,date,city,zip_code,quality_label,quality_code,no2,so2,o3,pm10,pm2-5,year,month,day,weekday,week_of_year
8859,2022-01-01,allennes les marais,59251,moyen,2,1,1,2,2,2,2022,1,1,5,52
199,2022-01-01,annoeullin,59112,moyen,2,1,1,2,2,2,2022,1,1,5,52
10669,2022-01-01,anstaing,59152,moyen,2,2,1,2,2,2,2022,1,1,5,52
11782,2022-01-01,armentieres,59280,moyen,2,1,1,2,2,2,2022,1,1,5,52
200,2022-01-01,aubers,59249,moyen,2,1,1,2,2,2,2022,1,1,5,52


#### 2.6 Multi-indexing

In [177]:
df = df.set_index(['city', 'date']).sort_index()
df

zip_code quality_label  quality_code  no2  \
city                date                                                    
allennes les marais 2022-01-01     59251         moyen             2    1   
                    2022-01-02     59251         moyen             2    1   
                    2022-01-03     59251         moyen             2    1   
                    2022-01-04     59251         moyen             2    1   
                    2022-01-05     59251         moyen             2    1   
...                                  ...           ...           ...  ...   
willems             2022-07-03     59780         moyen             2    1   
                    2022-07-04     59780         moyen             2    1   
                    2022-07-05     59780         moyen             2    2   
                    2022-07-06     59780         moyen             2    1   
                    2022-07-07     59780         moyen             2    1   

                                so2  o3  pm10  pm2-5  year  month  day  \
city                date                                                 
allennes les marais 2022-01-01    1   2     2      2  2022      1    1   
                    2022-01-02    1   2     2      2  2022      1    2   
                    2022-01-03    1   2     1      1  2022      1    3   
                    2022-01-04    1   2     1      1  2022      1    4   
                    2022-01-05    1   2     1      1  2022      1    5   
...                             ...  ..   ...    ...   ...    ...  ...   
willems             2022-07-03    1   2     1      1  2022      7    3   
                    2022-07-04    1   2     1      1  2022      7    4   
                    2022-07-05    1   2     1      1  2022      7    5   
                    2022-07-06    1   2     1      1  2022      7    6   
                    2022-07-07    1   2     1      1  2022      7    7   

                                weekday  week_of_year  
city                date                               
allennes les marais 2022-01-01        5            52  
                    2022-01-02        6            52  
                    2022-01-03        0             1  
                    2022-01-04        1             1  
                    2022-01-05        2             1  
...                                 ...           ...  
willems             2022-07-03        6            26  
                    2022-07-04        0            27  
                    2022-07-05        1            27  
                    2022-07-06        2            27  
                    2022-07-07        3            27  

[17500 rows x 13 columns]

#### 2.7 Exporting df to CSV

In [180]:
# generating a subset csv workfile with timestamp including in the name
timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
file_name = "!all_air-quality-index_" + timestr + ".csv"
print(f"file name: {file_name}")

df.to_csv(data_out + file_name, index = True)

file name: !all_air-quality-index_2022-10-07_16-44-31.csv


In [191]:

for city, df_city in df.groupby('city'):
    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    file_name = city + "_air-quality-index_" + timestr + ".csv"
    df_city.to_csv(data_out + file_name, index = True)

In [184]:
df.loc[('lille'), :]

,zip_code,quality_label,quality_code,no2,so2,o3,pm10,pm2-5,year,month,day,weekday,week_of_year
date,,,,,,,,,,,,,
2022-01-01,59000,moyen,2,2,1,2,2,2,2022,1,1,5,52
2022-01-02,59000,moyen,2,2,1,2,2,2,2022,1,2,6,52
2022-01-03,59000,moyen,2,2,1,2,1,1,2022,1,3,0,1
2022-01-04,59000,moyen,2,2,1,2,1,2,2022,1,4,1,1
2022-01-05,59000,moyen,2,2,1,2,1,1,2022,1,5,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-03,59000,moyen,2,1,1,2,1,1,2022,7,3,6,26
2022-07-04,59000,moyen,2,2,1,2,1,1,2022,7,4,0,27
2022-07-05,59000,moyen,2,2,1,2,1,2,2022,7,5,1,27


In [187]:
df.loc[('lambersart'), ['zip_code', 'quality_label', 'quality_code', 'month']].groupby('month').aggregate('mean')

,zip_code,quality_code
month,,
1,59130.0,2.666667
2,59130.0,2.000000
3,59130.0,3.103448
4,59130.0,2.166667
5,59130.0,2.225806
6,59130.0,2.448276
7,59130.0,2.000000


In [188]:
df.loc[(slice(None), '2022-03-29'), 'quality_label']

city                 date      
allennes les marais  2022-03-29         mauvais
annoeullin           2022-03-29    très mauvais
anstaing             2022-03-29         mauvais
armentieres          2022-03-29         mauvais
aubers               2022-03-29    très mauvais
                                       ...     
wattrelos            2022-03-29         mauvais
wavrin               2022-03-29    très mauvais
wervicq sud          2022-03-29         mauvais
wicres               2022-03-29    très mauvais
willems              2022-03-29         mauvais
Name: quality_label, Length: 95, dtype: object

In [199]:
df_unstacked = df.unstack(level='city')
df_unstacked.drop('zip_code', axis=1, inplace=True)
df_unstacked


quality_label                                           \
city       allennes les marais annoeullin anstaing armentieres   aubers   
date                                                                      
2022-01-01               moyen      moyen    moyen       moyen    moyen   
2022-01-02               moyen      moyen    moyen       moyen    moyen   
2022-01-03               moyen      moyen    moyen       moyen    moyen   
2022-01-04               moyen      moyen    moyen       moyen    moyen   
2022-01-05               moyen      moyen    moyen       moyen    moyen   
...                        ...        ...      ...         ...      ...   
2022-07-04               moyen      moyen    moyen       moyen    moyen   
2022-07-05               moyen      moyen    moyen       moyen    moyen   
2022-07-06               moyen      moyen    moyen       moyen    moyen   
2022-07-07               moyen      moyen    moyen       moyen    moyen   
2022-07-08             dégradé    dégradé    moyen     dégradé  dégradé   

                                                                    ...  \
city       baisieux   bassee   bauvin beaucamps ligny bois grenier  ...   
date                                                                ...   
2022-01-01    moyen    moyen    moyen           moyen        moyen  ...   
2022-01-02    moyen    moyen    moyen           moyen        moyen  ...   
2022-01-03    moyen    moyen    moyen           moyen        moyen  ...   
2022-01-04    moyen    moyen    moyen           moyen        moyen  ...   
2022-01-05    moyen    moyen    moyen           moyen        moyen  ...   
...             ...      ...      ...             ...          ...  ...   
2022-07-04    moyen    moyen    moyen           moyen        moyen  ...   
2022-07-05    moyen    moyen    moyen           moyen        moyen  ...   
2022-07-06    moyen    moyen    moyen           moyen        moyen  ...   
2022-07-07    moyen    moyen    moyen           moyen        moyen  ...   
2022-07-08    moyen  dégradé  dégradé         dégradé      dégradé  ...   

                week_of_year                                            \
city       villeneuve d ascq wambrechies warneton wasquehal wattignies   
date                                                                     
2022-01-01                52          52       52        52         52   
2022-01-02                52          52       52        52         52   
2022-01-03                 1           1        1         1          1   
2022-01-04                 1           1        1         1          1   
2022-01-05                 1           1        1         1          1   
...                      ...         ...      ...       ...        ...   
2022-07-04                27          27       27        27         27   
2022-07-05                27          27       27        27         27   
2022-07-06                27          27       27        27         27   
2022-07-07                27          27       27        27         27   
2022-07-08                27        <NA>     <NA>      <NA>       <NA>   

                                                        
city       wattrelos wavrin wervicq sud wicres willems  
date                                                    
2022-01-01        52     52          52     52      52  
2022-01-02        52     52          52     52      52  
2022-01-03         1      1           1      1       1  
2022-01-04         1      1           1      1       1  
2022-01-05         1      1           1      1       1  
...              ...    ...         ...    ...     ...  
2022-07-04        27     27          27     27      27  
2022-07-05        27     27          27     27      27  
2022-07-06        27     27          27     27      27  
2022-07-07        27     27          27     27      27  
2022-07-08      <NA>   <NA>        <NA>   <NA>    <NA>  

[185 rows x 1140 columns]

## 3. Data Analysis & Visualization

In [55]:
# Bon (1) > Moyen (2) > Dégradé (3) > Mauvais (4) > Très mauvais (5) > Extrêmement mauvais (6)  
df['quality_label'].value_counts()

moyen           12698
dégradé          2388
mauvais          2350
très mauvais       46
bon                18
Name: quality_label, dtype: int64

In [58]:
df[df['quality_code'] >= 5 ]

,city,zip_code,quality_label,quality_code,no2,so2,o3,pm10,pm2-5,year,month,day,weekday
date,,,,,,,,,,,,,
2022-01-14,aubers,59249,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,baisieux,59780,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,beaucamps ligny,59134,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,bondues,59910,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,chereng,59152,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,comines,59560,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,croix,59170,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,erquinghem le sec,59320,très mauvais,5,2,1,1,4,5,2022,1,14,4
2022-01-14,fromelles,59249,très mauvais,5,2,1,1,4,5,2022,1,14,4


In [64]:
df.groupby('weekday').mean()

,zip_code,quality_code,no2,so2,o3,pm10,pm2-5,year,month,day
weekday,,,,,,,,,,
0,59341.726316,2.318129,1.469396,1.0,1.999610,1.360234,1.753606,2022.0,3.592593,16.037037
1,59341.726316,2.500975,1.424561,1.0,1.958285,1.581676,1.921248,2022.0,3.666667,14.851852
2,59341.726316,2.652242,1.372710,1.0,2.114620,1.490838,1.906823,2022.0,3.703704,14.703704
3,59341.726316,2.474464,1.306043,1.0,2.201170,1.454191,1.749708,2022.0,3.703704,15.703704
4,59342.037161,2.311900,1.433820,1.0,2.103549,1.637578,1.683925,2022.0,3.707724,14.227557
5,59341.726316,2.264842,1.287579,1.0,2.004211,1.418526,1.589474,2022.0,3.480000,14.600000
6,59341.726316,2.340486,1.295142,1.0,2.084211,1.431174,1.738866,2022.0,3.692308,14.730769


In [71]:
df.groupby(by='weekday').aggregate('mean')

,zip_code,quality_code,no2,so2,o3,pm10,pm2-5,year,month,day,week_of_year
weekday,,,,,,,,,,,
0,59341.726316,2.318129,1.469396,1.0,1.999610,1.360234,1.753606,2022.0,3.592593,16.037037,14.0
1,59341.726316,2.500975,1.424561,1.0,1.958285,1.581676,1.921248,2022.0,3.666667,14.851852,14.0
2,59341.726316,2.652242,1.372710,1.0,2.114620,1.490838,1.906823,2022.0,3.703704,14.703704,14.0
3,59341.726316,2.474464,1.306043,1.0,2.201170,1.454191,1.749708,2022.0,3.703704,15.703704,14.0
4,59342.037161,2.311900,1.433820,1.0,2.103549,1.637578,1.683925,2022.0,3.707724,14.227557,13.672234
5,59341.726316,2.264842,1.287579,1.0,2.004211,1.418526,1.589474,2022.0,3.480000,14.600000,14.68
6,59341.726316,2.340486,1.295142,1.0,2.084211,1.431174,1.738866,2022.0,3.692308,14.730769,15.384615


In [74]:
df.groupby(by=['weekday', 'week_of_year']).aggregate({'quality_code': 'mean'})

quality_code
weekday week_of_year              
0       1                 2.000000
        2                 2.000000
        3                 3.000000
        4                 3.378947
        5                 2.000000
...                            ...
6       23                2.000000
        24                2.000000
        25                2.000000
        26                2.000000
        52                2.000000

[185 rows x 1 columns]

In [75]:
#The day of the week with Monday=0, Sunday=6.
df.groupby(by=['month', 'weekday']).aggregate({'quality_code': 'mean'})

quality_code
month weekday              
1     0            2.475789
      1            3.044737
      2            3.123684
      3            2.476316
      4            2.560526
      5            2.463158
      6            2.147368
2     0            2.005263
      1            2.000000
      2            2.015789
      3            2.000000
      4            2.000000
      5            2.000000
      6            2.000000
3     0            2.942105
      1            3.244211
      2            3.425263
      3            3.094737
      4            2.666667
      5            2.666667
      6            3.186842
4     0            2.105263
      1            2.281579
      2            2.055263
      3            2.573684
      4            2.002105
      5            2.000000
      6            2.339474
5     0            2.400000
      1            2.200000
      2            2.250000
      3            2.021053
      4            2.018421
      5            2.250000
      6            2.431579
6     0            2.000000
      1            2.250000
      2            2.941053
      3            2.610526
      4            2.842105
      5            2.435088
      6            2.000000
7     0            2.000000
      1            2.000000
      2            2.000000
      3            2.000000
      4            2.139130
      5            2.000000
      6            2.000000

In [ ]:
city_groups.mean()

,zip_code,quality_code,no2,so2,o3,pm10,pm2-5,year,month,day,weekday,week_of_year
city,,,,,,,,,,,,
allennes les marais,59251.0,2.405405,1.227027,1.0,2.081081,1.421622,1.713514,2022.0,3.664865,14.962162,2.956757,14.297297
annoeullin,59112.0,2.416216,1.227027,1.0,2.091892,1.421622,1.718919,2022.0,3.664865,14.962162,2.956757,14.297297
anstaing,59152.0,2.410811,1.367568,1.0,2.075676,1.421622,1.740541,2022.0,3.664865,14.962162,2.956757,14.297297
armentieres,59280.0,2.394595,1.237838,1.0,2.064865,1.481081,1.745946,2022.0,3.664865,14.962162,2.956757,14.297297
aubers,59249.0,2.394595,1.194595,1.0,2.070270,1.443243,1.729730,2022.0,3.664865,14.962162,2.956757,14.297297
...,...,...,...,...,...,...,...,...,...,...,...,...
wattrelos,59150.0,2.467391,1.375000,1.0,2.119565,1.510870,1.809783,2022.0,3.646739,15.000000,2.951087,14.228261
wavrin,59136.0,2.407609,1.271739,1.0,2.059783,1.451087,1.744565,2022.0,3.646739,15.000000,2.951087,14.228261
wervicq sud,59117.0,2.407609,1.255435,1.0,2.070652,1.472826,1.755435,2022.0,3.646739,15.000000,2.951087,14.228261


In [ ]:
#best and worst cities (low is good)
df.groupby('city').quality_code.mean().sort_values()

In [82]:
# bin histogram with quality label 

In [ ]:
# bin histogram with quality label for a given city versus another

In [ ]:
# number of days where air quality was bad (degradé+)

In [ ]:
#% od days where air quality was bad

In [ ]:
# check data per month and per weekday
